In [12]:
import requests
import os
import json
import pandas as pd

API_ENDPOINT="https://api.osmosis.interbloc.org/osmosis/gamm/v1beta1/pools/{p}"

UPGRADE_HEIGHT=4707300
HALT_HEIGHT=4713064

## Download all the data 

Pools affected by the bug taken from `https://docs.google.com/spreadsheets/d/1-0yiYnZhoeGvv1-u625KW7HV2y9KWGZKyoSH9zmBwIw/edit#gid=1119049066`

In [ ]:
POOLS = [
    1,
    2,
    3,
    4,
    5,
    7,
    9,
    10,
    15,
    42,
    463,
    481,
    482,
    497,
    498,
    553,
    557,
    560,
    561,
    572,
    573,
    574,
    577,
    578,
    580,
    584,
    585,
    597,
    600,
    601,
    602,
    604,
    605,
    606,
    608,
    610,
    611,
    613,
    617,
    618,
    619,
    621,
    625,
    627,
    629,
    631,
    633,
    637,
    638,
    641,
    643,
    648,
    649,
    651,
    653,
    662,
    669,
    674,
    678,
    681,
    690,
    701,
    704,
    712,
    716,
    719,
    720,
    721,
    722,
    730
]

Call made:

```
curl \
    -X GET \
    -H "Content-Type: application/json" \
    -H "x-cosmos-block-height: 4707300" \
    "https://api.osmosis.interbloc.org/osmosis/gamm/v1beta1/pools/1"
```

In [13]:
POOL_FILE_PATH = 'data/pools/{p}/'
FILE_PATH = 'data/pools/{p}/{h}.json'

POOLS = [1, 678, 712]

def get_pool_data(pool_id):

    for height in range(UPGRADE_HEIGHT, HALT_HEIGHT):
        print("pool:", pool_id, "height:", height, end='\r')

        if not os.path.exists(FILE_PATH.format(p=pool_id, h=height)):

            headers={
                'Accept': 'application/json',
                'x-cosmos-block-height': str(height),
            }
            response = requests.get(API_ENDPOINT.format(p=pool_id), headers=headers)

            with open(FILE_PATH.format(p=pool_id, h=height), 'w') as f:
                json.dump(response.json(), f)


for pool in POOLS:
    if not os.path.exists(POOL_FILE_PATH.format(p=pool)):
        os.makedirs(POOL_FILE_PATH.format(p=pool))

    get_pool_data(pool)

    

## Filter the Data

#### Get the downloaded data

In [ ]:
POOLS = [1]

results = []
for pool in POOLS:
    for height in range(UPGRADE_HEIGHT, HALT_HEIGHT):

        with open(FILE_PATH.format(h=height, p=pool)) as f:
            data = json.load(f)
        
        data["pool"]["height"] = height
        results.append(data["pool"])

#### Transform the DataFrame

In [ ]:
df = pd.json_normalize(results)

df = df.join(pd.json_normalize(pd.json_normalize(df["poolAssets"])[0]).add_prefix("token0_"))
df = df.join(pd.json_normalize(pd.json_normalize(df["poolAssets"])[1]).add_prefix("token1_"))
df.drop(columns=["poolAssets"], inplace=True)

### Save the DataFrame

In [ ]:
df.to_csv("csv/pools.csv", index=None)